In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 30.1 MB/s 
     |████████████████████████████████| 163 kB 70.6 MB/s 
     |████████████████████████████████| 7.6 MB 60.5 MB/s 


# 代码复现
https://github.com/terrifyzhao/spo_extract

In [2]:
## train 代码
 
import json
from tqdm import tqdm   # 进度条
import os
import numpy as np
from transformers import BertTokenizer, AdamW
import torch
# from model import ObjectModel, SubjectModel
 
GPU_NUM = 0
 
#device = torch.device(f'cuda:{GPU_NUM}') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')
 
vocab = {}
with open('vocab.txt')as file:  # 使用with open 的方法读取词典
    for l in file.readlines():
        vocab[len(vocab)] = l.strip() # 根据key读取词典
 
 
def load_data(filename):  # 中文解码加载数据
    """加载数据
    单条格式：{'text': text, 'spo_list': [[s, p, o]]}
    """
    with open(filename) as f:
        json_list = json.load(f)
    return json_list[:1000]
 
 
# 加载数据集
train_data = load_data('train_5494.json')
valid_data = load_data('dev_clean.json')

In [3]:
train_data[85],len(train_data),valid_data[85],len(valid_data)

({'text': 'My Man Godfrey is a 1936 American screwball comedy film directed by Gregory La Cava and starring William Powell and Carole Lombard, who had been briefly married years before appearing together in the film.',
  'spo_list': [['My Man Godfrey', 'act', 'William Powell']]},
 1000,
 {'text': 'Stagecoach War is a 1940 American Western film directed by Lesley Selander, written by Norman Houston and Harry F. Olmsted, and starring William Boyd, Russell Hayden, Julie Carter, Harvey Stephens, J. Farrell MacDonald, Britt Wood and Rad Robinson.',
  'spo_list': [['Stagecoach War', 'act', 'Rad Robinson']]},
 112)

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")   # 调用分词器
 
with open('schemas.json') as f: # 读取predicate
    json_list = json.load(f)
    id2predicate = json_list[0]
    predicate2id = json_list[1]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
id2predicate,predicate2id,len(id2predicate),len(predicate2id)

({'0': 'unknown', '1': 'direct', '2': 'act', '3': 'star'},
 {'unknown': 0, 'direct': 1, 'act': 2, 'star': 3},
 4,
 4)

In [4]:
from transformers import BertModel, BertPreTrainedModel

import torch.nn as nn
import torch


class SubjectModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dense = nn.Linear(config.hidden_size, 2)

    def forward(self, input_ids, attention_mask=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        # print(output)
        subject_out = self.dense(output[0])
        subject_out = torch.sigmoid(subject_out)
        # print("subject shape")
        # test = torch.sum(subject_out, dim=1)
        # print(test.shape, test)
        # print(subject_out)

        return output[0], subject_out


class ObjectModel(nn.Module):
    def __init__(self, subject_model):
        super().__init__()
        self.encoder = subject_model
        self.dense_subject_position = nn.Linear(2, 768)
        self.dense_object = nn.Linear(768, 4 * 2)

    def forward(self,input_ids,subject_position,attention_mask=None):
        # print("ObjectModel")
        output, subject_out = self.encoder(input_ids, attention_mask)
        # print(output, subject_out)
        subject_position = self.dense_subject_position(subject_position).unsqueeze(1)
        # print(subject_position)
        object_out = output + subject_position
        # print(object_out)
        # [bs, 768] -> [bs, 98]
        object_out = self.dense_object(object_out)
        # print(object_out)
        # [bs, 98] -> [bs, 4, 2]
        object_out = torch.reshape(object_out, (object_out.shape[0], object_out.shape[1], 4, 2))
        object_out = torch.sigmoid(object_out)
        # print("object_out")
        # test = torch.sum(object_out, dim=1)
        # print(test.shape, test)
        return subject_out, object_out

In [5]:
def search(pattern, sequence):
    """从sequence中寻找子串pattern
    如果找到，返回第一个下标；否则返回-1。
    """
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1
 
 
def sequence_padding(inputs, length=None, padding=0, mode='post'):
    """Numpy函数，将序列padding到同一长度
    """
    if length is None:
        length = max([len(x) for x in inputs])
 
    pad_width = [(0, 0) for _ in np.shape(inputs[0])]
    outputs = []
    for x in inputs:
        x = x[:length]
        if mode == 'post':
            pad_width[0] = (0, length - len(x))
        elif mode == 'pre':
            pad_width[0] = (length - len(x), 0)
        else:
            raise ValueError('"mode" argument must be "post" or "pre".')
        x = np.pad(x, pad_width, 'constant', constant_values=padding)
        outputs.append(x)
 
    return np.array(outputs)

In [15]:
def data_generator(data, batch_size=3):  #  数据迭代器/数据生成器
 
    batch_input_ids, batch_attention_mask = [], [] #  输出给模型（object）的变量，通过调用bert分词器得到
    batch_subject_labels, batch_subject_ids, batch_object_labels = [], [], []
    texts = []
    for i, d in enumerate(data): #  数据来自dataloader i = 数据索引 d = text
        # print("______________data_generator___________") 1 
        # print(d)  2
        text = d['text'] # 从train 中取出text
        texts.append(text)   # text 贴入元组
        encoding = tokenizer(text=text) # 使用bert 分词
        input_ids, attention_mask = encoding.input_ids, encoding.attention_mask  # 分词后对应“bert词典下标”和mask
        # 整理三元组 {s: [(o, p)]}
        spoes = {}
        for s, p, o in d['spo_list']: # 遍历三元组
            # [cls] XXX [sep]
            s_encoding = tokenizer(text=s).input_ids[1:-1]  # 将s，o编码成对应的下标
            o_encoding = tokenizer(text=o).input_ids[1:-1]  # [1:-1] 去除cls sep
 
            s_idx = search(s_encoding, input_ids) # 从text的input_ids 寻找s的下标
            o_idx = search(o_encoding, input_ids) # 从text的input_ids 寻找o的下标
            # print("s", s, s_encoding, s_idx) 3
            # print("o", o, o_encoding, o_idx) 4
 
            p = predicate2id[p]  # 的到predicate的下标
 
            if s_idx != -1 and o_idx != -1: # 做判断没有反应的返回-1
                s = (s_idx, s_idx + len(s_encoding) - 1) # s保存subject的起始位置，起始位置加上长度 -1
                o = (o_idx, o_idx + len(o_encoding) - 1, p)# 同上 s,o 是一个元组保存着起始位置和终止位置的下标 以及 p
                if s not in spoes:
                    spoes[s] = []
                spoes[s].append(o) # 将 下标加入 spoes 字典当中去
        # print("input_ids", len(input_ids), input_ids) 5
        # print("spoes:", spoes) 6
        if spoes:
            # subject标签
            subject_labels = np.zeros((len(input_ids), 2)) # 生成一个input长度的二维向量/ s头s尾
            # print(input_ids, subject_labels, subject_labels.shape) 7
            for s in spoes:
                # 注意要+1，因为有cls符号
                subject_labels[s[0], 0] = 1 # 第一行 = ‘0’ 的起始 = s[0] 等于1
                subject_labels[s[1], 1] = 1  # 第二行 = ‘1’ 的终止 =s[1] 等于1
            # print("subject_labels:", subject_labels) 7
            # 一个s对应多个o时，随机选一个subject
            start, end = np.array(list(spoes.keys())).T
            start = np.random.choice(start)
            end = np.random.choice(end[end >= start])
            subject_ids = (start, end)
            # 对应的object标签
            object_labels = np.zeros((len(input_ids), len(predicate2id), 2)) # 序列长度 x predicate长度 x 2
            for o in spoes.get(subject_ids, []): # 通过subject 拿出对应的 o
                object_labels[o[0], o[2], 0] = 1 # 对应 起始位置，predicate ， 第一维度/头（取字o元组）
                object_labels[o[1], o[2], 1] = 1 # 同上
            # 构建batch
            batch_input_ids.append(input_ids)  # 将上述值加入batch
            batch_attention_mask.append(attention_mask)
            batch_subject_labels.append(subject_labels)
            batch_subject_ids.append(subject_ids)
            batch_object_labels.append(object_labels)
            if len(batch_subject_labels) == batch_size or i == len(data) - 1: # 没有补偿
                batch_input_ids = sequence_padding(batch_input_ids)
                batch_attention_mask = sequence_padding(batch_attention_mask)
                batch_subject_labels = sequence_padding(batch_subject_labels)
                batch_subject_ids = np.array(batch_subject_ids)
                batch_object_labels = sequence_padding(batch_object_labels)
                yield [
                          torch.from_numpy(batch_input_ids).long(), torch.from_numpy(batch_attention_mask).long(),
                          torch.from_numpy(batch_subject_labels), torch.from_numpy(batch_subject_ids),
                          torch.from_numpy(batch_object_labels)
                      ], None
                batch_input_ids, batch_attention_mask = [], [] # 清空进入下个batch
                batch_subject_labels, batch_subject_ids, batch_object_labels = [], [], []
 
 
if os.path.exists('graph_model.bin'):  # 加载模型 保存档将graph model 加载过来
    print('load model')
    model = torch.load('graph_model.bin').to(device)
    subject_model = model.encoder
else:

    #subject_model = SubjectModel.from_pretrained('facebook/bart-base') # 没有使用bert train
    subject_model = SubjectModel.from_pretrained('bert-base-cased') # 没有使用bert train
    subject_model.to(device)
 
    model = ObjectModel(subject_model)
    model.to(device)
 
train_loader = data_generator(train_data, batch_size=8) # dataloader = 8
 
optim = AdamW(model.parameters(), lr=5e-10) # 加速器 adamw 学习率 5e-5
loss_func = torch.nn.BCELoss() # cross binary loss
 
model.train()
# print(model.parameters())

Some weights of the model checkpoint at bert-base-cased were not used when initializing SubjectModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing SubjectModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SubjectModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SubjectModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['dense.bias', 'dense.weig

ObjectModel(
  (encoder): SubjectModel(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
              

In [16]:
class SPO(tuple):
    def __init__(self, spo):
        self.spox = (
            spo[0],
            spo[1],
            spo[2],
        )
 
    def __hash__(self):
        return self.spox.__hash__()
 
    def __eq__(self, spo):
        return self.spox == spo.spox
 
 
def train_func():
    train_loss = 0
    pbar = tqdm(train_loader) # 开启进度条并遍历 train_loader
    for step, batch in enumerate(pbar): # 遍历每个step 和 batch
        print(step,'step')
        optim.zero_grad()  # 将每个梯度清零
        batch = batch[0] # 将batch 数据取出来第一个维度
        
        input_ids = batch[0].to(device) # text对应bert词典的下标
        # print(input_ids,'input_ids') 1
        attention_mask = batch[1].to(device) # mask
        # print(attention_mask,'attention_mask') 2
        subject_labels = batch[2].to(device) # subject对应bert词典的下标
        subject_ids = batch[3].to(device) # subject 在句子中id
        object_labels = batch[4].to(device) # object对应bert词典的下标
 
        subject_out, object_out = model(input_ids, subject_ids.float(), attention_mask) # 拿到subject和object输出
        subject_out = subject_out * attention_mask.unsqueeze(-1) # 将输入中补长的位置变成 0 / input当中的padding
        object_out = object_out * attention_mask.unsqueeze(-1).unsqueeze(-1)#  同上

        # print("subject_loss:", subject_out.shape, subject_out, subject_labels.shape, subject_labels) 3
        subject_loss = loss_func(subject_out, subject_labels.float()) # 识别subject的损失函数
        # print("object_loss:", object_out.shape, object_out, object_labels.shape, object_labels) 4
        object_loss = loss_func(object_out, object_labels.float()) #  识别object的损失函数
 
        # subject_loss = torch.mean(subject_loss, dim=2)
        # subject_loss = torch.sum(subject_loss * attention_mask) / torch.sum(attention_mask)
 
        loss = subject_loss + object_loss # 将loss进行相加 根据实际情况添加超参数
 
        train_loss += loss.item() # 累加到train loss
        loss.backward() # 反向传播
        optim.step() # 更新参数
 
        pbar.update()
        pbar.set_description(f'train loss:{loss.item()}')  # 显示更新参数
 
        if step % 1000 == 0:  # 每跑1000个step 保存模型
            torch.save(model, 'graph_model.bin')
 
        if step % 1 == 0 and step != 0:  # 每跑100步在验证集当中检验效果
            with torch.no_grad():
                # texts = ['The film stars William Boyd, Russell Hayden, Andy Clyde, Eleanor Stewart, Morris Ankrum and William Haade.',
                #          'Texas Rangers Ride Again is a 1940 American Western film directed by James P. Hogan, written by William R. Lipman and Horace McCoy, and starring Ellen Drew, John Howard, Akim Tamiroff, May Robson, Broderick Crawford, Charley Grapewin, and John Miljan.']
                X, Y, Z = 1e-10, 1e-10, 1e-10
                pbar = tqdm()
                spo = []
                for data in valid_data[0:1]: # 遍历验证集
                # for text in texts:
                    text = data['text'] # 取出text
                    spo_ori = data['spo_list'] # 去除三元组
                    en = tokenizer(text=text, return_tensors='pt') # 将text分词
                    _, subject_preds = subject_model(en.input_ids.to(device), en.attention_mask.to(device)) # 检验阶段需要预测subject的下标
                    subject_preds = subject_preds.cpu().data.numpy() # 将下标转换成numpy数组
                    # print("validate: subject_preds")
                    # print(subject_preds)
                    start = np.where(subject_preds[0, :, 0] > 0.5)[0] # 阈值，大于0.5判断为start
                    end = np.where(subject_preds[0, :, 1] > 0.4)[0] # 阈值 大于0.4判断为end # 阈值自己设定
                    # print(start, end)
                    subjects = []
                    for i in start: # 遍历start 用来应对多个start的情况
                        i_end = [end[j] for j in range(len(end)) if end[j] >= i and end[j] <= i+6] # 只取大于start的end 否则会出现逻辑错误
                        # print("i and j:")
                        # print(i_end)
                        if len(i_end)>0: # 如果 end 大于0将 start end 成对加入subject
                            for j in i_end:
                              subjects.append((i, j))
                    # print(subjects)
                    if subjects:
                        for s in subjects: # 遍历每个s
                            index = en.input_ids.cpu().data.numpy().squeeze(0)[s[0]:s[1] + 1] # 根据输入的下标
                            # print("subject index:")
                            # print(index)
                            subject = '' # 将bert的vcab里的汉字映射出来
                            for i in index:
                              v = vocab[i]
                              if i == 102:
                                continue
                              if v.startswith('##'):
                                v = v.split('#')[-1]
                                subject += v
                              else:
                                subject += ' ' + v
                            subject = subject.strip()
                            # print(subject)
                            _, object_preds = model(en.input_ids.to(device), # 将input分词的结果添加进去
                                                    torch.from_numpy(np.array([s])).float().to(device), # s的下标添加进去
                                                    en.attention_mask.to(device)) # 将mask添加进去
                            object_preds = object_preds.cpu().data.numpy() # 转换成numpy数组
                            # print("validate: object_preds")
                            # print("shape of object_preds________________________________________________")
                            # print(object_preds.shape)
                            for object_pred in object_preds:  # 遍历所有的object
                                # print(object_pred.shape)
                                start = np.where(object_pred[:, :, 0] > 0.4) # object的阈值大于0.2取start
                                end = np.where(object_pred[:, :, 1] > 0.2) # 同上
                                for _start, predicate1 in zip(*start): # 星号zip代表把两个值解开 两行对应的元组 # 遍历start取 s 和 p
                                    for _end, predicate2 in zip(*end): # 遍历end 取 e 和 p
                                        if _start <= _end and _end <= _start+6 and predicate1 == predicate2: # 判断是否复合逻辑 spo
                                            index = en.input_ids.cpu().data.numpy().squeeze(0)[_start:_end + 1] # 从输入中找到对应下标
                                            # the_object = ' '.join([vocab[i] for i in index]) # 从bert词典中映射成中文
                                            the_object = '' # 将bert的vcab里的汉字映射出来
                                            for i in index:
                                              v = vocab[i]
                                              if i == 102:
                                                continue
                                              if v.startswith('##'):
                                                v = v.split('#')[-1]
                                                the_object += v
                                              else:
                                                the_object += ' ' + v
                                            the_object = the_object.strip()
                                            predicate = id2predicate[str(predicate1)] # 找predicate下标返回predicate
                                            # print(object, '\t', predicate)
                                            spo.append([subject, predicate, the_object])  # 三元组放到数组当中
                    # 预测结果
                    # print("Calculate the score:")
                    R = set([SPO(_spo) for _spo in spo]) # 预测去重
                    # print('R:', R)
                    # 真实结果
                    T = set([SPO(_spo) for _spo in spo_ori]) # 真是去重
                    # print('T:', T)
                    print('R and T:', R.intersection(T))
                    # R = set(spo_ori)
                    # T = set(spo)
                    # 交集
                    X += len(R.intersection(T)) #& T R & T 交集长度
                    Y += len(R)  # R 长度 预测结果个数
                    Z += len(T)#=1 # T 长度 真实结果个数
                    f1, precision, recall = 2 * X / (Y + Z), X / Y, X / Z # f1 精准度 召回率
                    pbar.update()  # 把代码更新到pbar
                    pbar.set_description(
                        'f1: %.5f, precision: %.5f, recall: %.5f' % (f1, precision, recall)
                    )
                # pbar.close()
                print('f1:', f1, 'precision:', precision, 'recall:', recall)
 
 
for epoch in range(20):
    print('************start train************')
    train_func()

************start train************


0it [00:00, ?it/s]

0 step


train loss:1.0569047927856445: : 2it [00:04,  2.16s/it]

1 step


...

R and T: set()
f1: 1.1047892614483668e-14 precision: 5.524251463926608e-15 recall: 9.999999999e-11
18 step



f1: 0.00000, precision: 0.00000, recall: 0.00000: : 2it [00:09,  4.27s/it]
train loss:1.380097508430481: : 2it [00:09,  4.27s/it]                    

train loss:1.380097508430481: : 2it [00:09,  4.56s/it]


1it [00:04,  4.08s/it]

train loss:0.9928063154220581: : 19it [02:59,  9.11s/it]

R and T: set()
f1: 1.510231820584437e-14 precision: 7.551729346020182e-15 recall: 9.999999999e-11
19 step




f1: 0.00000, precision: 0.00000, recall: 0.00000: : 2it [00:07,  3.62s/it]

train loss:2.721737861633301: : 2it [00:07,  3.62s/it]                    
train loss:2.721737861633301: : 2it [00:07,  3.70s/it]
train loss:0.9928063154220581: : 19it [03:04,  9.72s/it]


KeyboardInterrupt: ignored